## Create a generative AI runbook to resolve security findings

## Module 1

```
// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0
```

### Introduction

In this module, you will complete the following
- Get a finding from Security Hub
- Use Claude 3 to summarize the finding result
- Use Claude 3 to generate Python code used for finding discovery
- Use Claude 3 to generate Python code used for remediating the finding

**NOTE**: You can use SHIFT + ENTER to run each of the cells in this workbook.

**In the first example, get a failed finding from Security Hub for `Security groups should not allow unrestricted access to ports with high risk [EC2.19]`**

#### Step 1: Get a finding from Security Hub

In this step, you will get a failed finding from Security Hub. In this example, we use the AWS SDK for Python (boto3) to access the Security Hub API.

The Python function will return a JSON representation of the finding. You can also view this finding by visiting Security Hub in the AWS console.

You can safely ignore `Note: you may need to restart the kernel to use updated packages.`

In [ ]:
%pip install -qU pip boto3 awscli botocore
import boto3

sh = boto3.client('securityhub')

def get_finding(finding_id):
    response = sh.get_findings(
        Filters={
                'ComplianceSecurityControlId': [
                    {
                        'Value': finding_id,
                        'Comparison': 'EQUALS'
                    }
                ],
                'ComplianceStatus':[
                    {
                        'Value': 'FAILED',
                        'Comparison': 'EQUALS'
                    }
                ],
                'ResourceTags': [
                    {
                        'Key': 'Workshop',
                        'Value': 'SampleSecurityGroup',
                        'Comparison': 'EQUALS'
                    }
                ]
            },
        MaxResults=1

    )
    return response['Findings']

sg_finding = get_finding("EC2.19")
print(sg_finding)

#### Step 2: Summarize the finding using Claude 3

In this step, you will use the Anthropic SDK, to access Claude 3 through Amazon Bedrock.

In [ ]:
import boto3

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

bedrock_client = boto3.client(service_name='bedrock-runtime')
inference_config = {"temperature": 0}

def get_completion(prompt, system_prompt=None):

    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config,
    }

    # Check if system_prompt is provided and add the system parameter to the converse_api_params dictionary
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]

    response = bedrock_client.converse(**converse_api_params)

    return response['output']['message']['content'][0]['text']

After loading the function, you can create your first prompt.

In [ ]:
system_prompt = "You are an AWS Security Engineer looking to improve the security posture of your organization."

prompt = f"""
    Review the finding and summarize actionable next steps

    <finding>
    {sg_finding}
    </finding>

"""

response = get_completion(prompt, system_prompt)

print(response)

#### Step 3 - Use Claude 3 to generate Python code used for finding discovery

Based on the output in step 2, Claude 3 recommends reviewing the finding and restrict inbound access to only trusted addresses or ranges. 

In this step, you will write a prompt to determine where this security group is being used.

`Ensure you update the prompt with the Security Group ID of the finding in step 2`

In [ ]:
# Update the prompt with the security group ID of your finding.
prompt = f"""

Write a simple Python function to determine if sg-0f1ec3a90fa6ae3be security group is being used by an ENI.

"""
print(get_completion(prompt))

Review the output. Claude 3 creates a Python function that takes a security group ID, and returns true if the security group is associated to a network interface (ENI). 

Create the function Claude 3 generated for checking if a security group is being used.

`Note: Make sure you update security_group_id with the security group output in step 2.`

In [ ]:
import boto3

def is_security_group_used_by_eni(security_group_id):
    """
    Checks if a given security group is being used by an Elastic Network Interface (ENI).

    Args:
        security_group_id (str): The ID of the security group to check.

    Returns:
        bool: True if the security group is being used by an ENI, False otherwise.
    """
    ec2 = boto3.client('ec2')

    # Get all network interfaces
    network_interfaces = ec2.describe_network_interfaces()['NetworkInterfaces']

    # Check if the security group is associated with any network interface
    for network_interface in network_interfaces:
        groups = [group['GroupId'] for group in network_interface['Groups']]
        if security_group_id in groups:
            return True

    return False

######
### Update security_group_id = "XXX" with the security group from step 2 output.
######

security_group_id = 'XXX'
is_used = is_security_group_used_by_eni(security_group_id)
print(f"Security group {security_group_id} is {'being used' if is_used else 'not being used'} by an ENI.")


Claude 3 returns that the security group is not in use. To test if the function is working properly, reviewing another security group that is attached to an ENI. In this workshop, we have provisioned a security group with the name `Group2` for testing. Run the following function to get the security group ID.

In [ ]:
#This function takes the name of the security group and returns the security group ID

import boto3
ec2 = boto3.client('ec2')
security_group_name = 'Group2'
response = ec2.describe_security_groups(
    Filters=[
        {
            'Name': 'group-name',
            'Values': [security_group_name]
        }
    ]
)

group2_id = response['SecurityGroups'][0]['GroupId']
print(group2_id)

With the security group ID for Group2, use the function to see if it is in use.

In [ ]:
is_used = is_security_group_used_by_eni(group2_id)
print(f"Security group {group2_id} is {'being used' if is_used else 'not being used'} by an ENI.")

Claude 3 returns that the security group is in use. Group2 findings, may require more research to understand how it is being used before making changes. **Note**: This workshop example is simplified by design. In a production environement, you should follow your change control process and do further diligence before making changes.

#### Step 4: Use Claude 3 to generate Python code used for remediating the finding

In this step, we will ask Claude 3 to generate remediation function to update the security group that is not being used and remove port 22.

In [ ]:
prompt = f"""Review the finding.

<finding>
{sg_finding}
</finding>

Create a Python function called remove_ssh_access_from_security_group to remove the rule for inbound access on port 22 (SSH) from any IP address.

You should only output the Python."""

print(get_completion(prompt))

Claude 3 will generate a function to remove port 22 from a security group ID. Carefully review the function. Once you are familiar with how it works, run the function to restrict access to the security group port 22.

In [ ]:
import boto3

def remove_ssh_access_from_security_group(security_group_id):
    ec2 = boto3.client('ec2')

    # Get the security group details
    security_group = ec2.describe_security_groups(GroupIds=[security_group_id])['SecurityGroups'][0]

    # Find the ingress rule for SSH from 0.0.0.0/0
    ssh_rule = next((rule for rule in security_group['IpPermissions'] if rule['FromPort'] == 22 and rule['ToPort'] == 22 and '0.0.0.0/0' in [ip_range['CidrIp'] for ip_range in rule['IpRanges']]), None)

    if ssh_rule:
        # Remove the SSH rule
        ec2.revoke_security_group_ingress(
            GroupId=security_group_id,
            IpPermissions=[{
                'IpProtocol': ssh_rule['IpProtocol'],
                'FromPort': ssh_rule['FromPort'],
                'ToPort': ssh_rule['ToPort'],
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
            }]
        )
        print(f"Removed SSH access from 0.0.0.0/0 for security group {security_group_id}")
    else:
        print(f"No SSH access from 0.0.0.0/0 found for security group {security_group_id}")

remove_ssh_access_from_security_group(security_group_id)

The rule will be removed from the security group. You can verify, by running the function again, or checking in the AWS console.